In [1]:
import yfinance as yf
from auxillary import *
from indicators import *

IndentationError: unindent does not match any outer indentation level (indicators.py, line 271)

In [16]:
stocks = []
print("Enter the ticker symbols of interest (enter spacebar to exit)")
s = "  "
while s != " ":
    s = input("Ticker: ")
    if s == " " and len(stocks) == 0:
        print("Please enter a ticker symbol")
        s = "  "
        continue
    elif s == " ":
        break
    if s.upper() not in stocks:
        stocks.append(s.upper())
    

n = int(input("How many trading days in the future would you like to predict? "))



Enter the ticker symbols of interest (enter spacebar to exit)
Please enter a ticker symbol
Please enter a ticker symbol
Please enter a ticker symbol


In [18]:
yf.download(stocks, period = "max")

[*********************100%***********************]  4 of 4 completed


Adj Close                                      \
                                 AAPL       GOOGL        MSFT         SPY   
Date                                                                        
1980-12-12 00:00:00-05:00    0.099722         NaN         NaN         NaN   
1980-12-15 00:00:00-05:00    0.094519         NaN         NaN         NaN   
1980-12-16 00:00:00-05:00    0.087582         NaN         NaN         NaN   
1980-12-17 00:00:00-05:00    0.089749         NaN         NaN         NaN   
1980-12-18 00:00:00-05:00    0.092351         NaN         NaN         NaN   
...                               ...         ...         ...         ...   
2023-03-31 00:00:00-04:00  164.899994  103.730003  288.299988  409.390015   
2023-04-03 00:00:00-04:00  166.169998  104.360001  287.230011  410.950012   
2023-04-04 00:00:00-04:00  165.630005  104.720001  287.179993  408.670013   
2023-04-05 00:00:00-04:00  163.759995  104.470001  284.339996  407.600006   
2023-04-06 00:00:00-04:00  164.660004  108.419998  291.600006  409.190002   

                                Close                                      \
                                 AAPL       GOOGL        MSFT         SPY   
Date                                                                        
1980-12-12 00:00:00-05:00    0.128348         NaN         NaN         NaN   
1980-12-15 00:00:00-05:00    0.121652         NaN         NaN         NaN   
1980-12-16 00:00:00-05:00    0.112723         NaN         NaN         NaN   
1980-12-17 00:00:00-05:00    0.115513         NaN         NaN         NaN   
1980-12-18 00:00:00-05:00    0.118862         NaN         NaN         NaN   
...                               ...         ...         ...         ...   
2023-03-31 00:00:00-04:00  164.899994  103.730003  288.299988  409.390015   
2023-04-03 00:00:00-04:00  166.169998  104.360001  287.230011  410.950012   
2023-04-04 00:00:00-04:00  165.630005  104.720001  287.179993  408.670013   
2023-04-05 00:00:00-04:00  163.759995  104.470001  284.339996  407.600006   
2023-04-06 00:00:00-04:00  164.660004  108.419998  291.600006  409.190002   

                                 High              ...         Low  \
                                 AAPL       GOOGL  ...        MSFT   
Date                                               ...               
1980-12-12 00:00:00-05:00    0.128906         NaN  ...         NaN   
1980-12-15 00:00:00-05:00    0.122210         NaN  ...         NaN   
1980-12-16 00:00:00-05:00    0.113281         NaN  ...         NaN   
1980-12-17 00:00:00-05:00    0.116071         NaN  ...         NaN   
1980-12-18 00:00:00-05:00    0.119420         NaN  ...         NaN   
...                               ...         ...  ...         ...   
2023-03-31 00:00:00-04:00  165.000000  103.889999  ...  283.000000   
2023-04-03 00:00:00-04:00  166.289993  104.529999  ...  283.950012   
2023-04-04 00:00:00-04:00  166.839996  105.580002  ...  285.670013   
2023-04-05 00:00:00-04:00  165.050003  106.099998  ...  282.920013   
2023-04-06 00:00:00-04:00  164.960007  109.169998  ...  282.029999   

                                             Open                          \
                                  SPY        AAPL       GOOGL        MSFT   
Date                                                                        
1980-12-12 00:00:00-05:00         NaN    0.128348         NaN         NaN   
1980-12-15 00:00:00-05:00         NaN    0.122210         NaN         NaN   
1980-12-16 00:00:00-05:00         NaN    0.113281         NaN         NaN   
1980-12-17 00:00:00-05:00         NaN    0.115513         NaN         NaN   
1980-12-18 00:00:00-05:00         NaN    0.118862         NaN         NaN   
...                               ...         ...         ...         ...   
2023-03-31 00:00:00-04:00  404.549988  162.440002  101.300003  283.730011   
2023-04-03 00:00:00-04:00  408.440002  164.270004  102.389999  286.519989   
2023-04-04 00:00:00-04:00  407.239990  166

In [ ]:
ticker = yf.Ticker("AAPL")
df = ticker.history(period = "max") 
df = adjustPrices(df)
df["Future_Price_Change"] = calculateFuturePriceChange(df,n)
df = createFeatures(df)

In [ ]:


df